In [80]:
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
import numpy as np
import cPickle

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

tf.logging.set_verbosity(tf.logging.INFO)

num_classes=10
dropout_rate=0.4
learning_rate=0.001
minibatch_size=100

def unpickle(file_name):
    with open(file_name, 'rb') as fo:
        data = cPickle.load(fo)
    return data

def cnn_model(features, labels, mode):
    input_data = tf.reshape(features, [-1, 3, 32, 32])
    input_layer = tf.to_float(input_data)
    
    conv1_layer = tf.layers.conv2d(
        inputs=input_layer,
        filters=40,
        kernel_size=(5,5),
        strides=(2,2),
        padding='same',
        data_format='channels_first',
        activation=tf.nn.relu)
    
    pool1_layer = tf.layers.max_pooling2d(
        inputs=conv1_layer,
        pool_size=(2,2),
        strides=(2,2),
        padding='same',
        data_format='channels_first')
    
    conv2_layer = tf.layers.conv2d(
        inputs=pool1_layer,
        filters=80,
        kernel_size=(3,3),
        strides=(1,1),
        padding='same',
        data_format='channels_first',
        activation=tf.nn.relu)
    
    pool2_layer = tf.layers.max_pooling2d(
        inputs=conv2_layer,
        pool_size=(2,2),
        strides=(2,2),
        padding='same',
        data_format='channels_first')
    
    pool2_flat = tf.reshape(pool2_layer, [-1, 80*4*4])
    dense1 = tf.layers.dense(pool2_flat, units=400, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
        inputs=dense1, 
        rate=dropout_rate, 
        training=mode==learn.ModeKeys.TRAIN)
    
    class_prediction = tf.layers.dense(inputs=dropout, units=num_classes)
    
    training_op = None
    loss = None
    if mode != learn.ModeKeys.INFER: # ground-truth is unknown if we are inferring with new examples
        ground_truth = tf.one_hot(indices=labels, depth=num_classes)
#        print(class_prediction)
        ground_truth = tf.reshape(ground_truth, [-1, 10])
#        print(ground_truth)
        loss = tf.losses.softmax_cross_entropy(onehot_labels=ground_truth,
                                       logits=class_prediction)
    
    if mode == learn.ModeKeys.TRAIN:
        training_op = tf.contrib.layers.optimize_loss(
            loss=loss, 
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=learning_rate,
            optimizer="SGD")
    
    predictions = {'classes': tf.argmax(input=class_prediction, axis=1), # the prediction
                   'probabilties': tf.nn.softmax(class_prediction, name='softmax') }# predicted probabilities
    
    return model_fn_lib.ModelFnOps(
        mode=mode, predictions=predictions, loss=loss, train_op=training_op)

In [68]:
data = unpickle('dataset/cifar-10-batches-py/data_batch_1')

In [78]:
test_index = 5
image = np.reshape(data['data'][test_index:test_index+minibatch_size], [-1, 3, 1024])
label = data['labels'][test_index:test_index+minibatch_size]
# image_reordered = []
#for i in range(1024):
#    image_reordered.append([image[0][i], image[1][i], image[2][i]])
#
#image_reordered = np.reshape(image_reordered, [32, 32, 3])
#plt.imshow(image_reordered, shape=(32,32), interpolation='bicubic')
#print label

In [81]:
cnn_model(image, label, mode=learn.ModeKeys.TRAIN)

Tensor("dense_48/BiasAdd:0", shape=(100, 10), dtype=float32)
Tensor("one_hot_18:0", shape=(100, 10), dtype=float32)


ModelFnOps(predictions={'probabilties': <tf.Tensor 'softmax_5:0' shape=(100, 10) dtype=float32>, 'classes': <tf.Tensor 'ArgMax_7:0' shape=(100,) dtype=int64>}, loss=<tf.Tensor 'softmax_cross_entropy_loss_11/value:0' shape=() dtype=float32>, train_op=<tf.Operation 'OptimizeLoss_7/control_dependency' type=Identity>, eval_metric_ops={}, output_alternatives=None, training_chief_hooks=[], training_hooks=[], training_scaffold=None)